## Model project - computable general equilibrium

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import ipywidgets as widgets
from types import SimpleNamespace
from scipy import linalg
from scipy import optimize
from IPython.display import display
from ipywidgets import interact, fixed
import modelproject as model_code_Final2
import importlib

In [2]:

%load_ext autoreload
%autoreload 2

In our project we are working with a model describing 2 different sectors. The model is similar to one considered in an old assignment from the 2017-version of the course "Anvendte Generelle Ligevægtsmodeller". One sector with high productivity growth and the other with lower productivity growth.
We look at how the different sectors choose inputs and simulate scenarios for different elasticities and how the difference in productivity growth affects the demand for inputs.
Finally we will add a growth rate for materials as an extension.

| Variable | Explanation || Variable | Explanation |
| --- | --- | | --- | --- | 
| $\Theta_j$ | productivity of labour || $\mu_{i,j}^{x}$ | capital intensity |
| $L_{j}$ | labour || $p_i$ | price of input i |
| $\mu_{j}^{YL}$ | scale parameter || $C_j$ | consumption demanded |
| $w$ | wage || $\gamma_j$ | scale parameter |
| $p_j$ | price for good j || $p_C$ | price index for consumption goods |
| $E^Y$ | elasticity of substitution between labour and materials || $x_{i,j}$ | inputs produced by sector i, consumed by sector j |
| $Y_j$ | real output || $E^C$ | elasticity of substitution between goods |
| $M_j$ | materials demanded  || $Y^D$ | output demanded |
| $p_j^M$ |  price index for materials || $N$ | population |
| $E^M$ |  Elasticity of substitution between material inputs ||







## The model

Labour Demand
1. $$\Theta_{j} L_{j} = \mu_{j}^{YL} \left( \frac{w}{\Theta_{j} p_{j}} \right) ^{-E^Y} Y_{j}$$

Material Aggregate demand

2. $$
M_{j} = \mu_{j}^{YM}  \left( \frac {p_{j}^{M}} {p_{j}} \right) ^{-E^Y} Y_{j}
$$

Zero-profit assumption

3. $$
p_{j}^{M} M_{j} + wL_{j} = p_{j}Y_{j}
$$

Division of aggregate materials

4. $$
x_{i,j} = \mu_{i, j}^{x} \left( \frac {p_{i}} {p_{j}^{M}} \right) ^{-E^M} M_{j}
$$


Implicit definition of aggregate material price

5. 
$$
p_{j}^{M} M_{j} = \sum_{i} x_{i,j} p_{i}
$$

Demand for products

6. $$
C_{j} = \gamma_{j}  \left( \frac {p_{j}} {p_{C}} \right) ^{-E^C} \frac {Y^{D}} {p_{C}}
$$

Budget constraint

7. 

$$
Y^{D} = \sum_{j} p_{j} C{j}
$$

Disposable income

8. $$
Y^{D} = wN
$$

Product market equilibrium

9. $$
Y_{i} =  \sum_{j} x_{i, j} + C_{i}
$$

The economy consists of two sectors both having a nested CES production-function. Both sectors use materials and labor in production. However, material inputs are themselves from a CES-production function determining demand for materials produced by themselves and in the other sector. The general form of CES production functions and CES utility is shown below:

CES Production Function 
$$
F(K,L) = ((\mu_1 (\Theta L)^{\frac{\sigma}{\sigma-1}}+\mu_2 M^{\frac{\sigma}{\sigma-1}})^{\frac{\sigma-1}{\sigma}}
$$

CES Utility Function
$$
U(C_1, C_2) = (( \gamma_1 C_1^{\frac{\rho}{\rho-1}} + \gamma_2 C_2^{\frac{\rho}{\rho-1}}  ))^{\frac{\rho¨-1}{\rho}}
$$
where $\sigma$ is elasticity of substitution between inputs in production and $\rho$ is elasticity of substitution between consumption goods. 

All input and consumption demand can be derived from these when using the budget constraints or total cost expressions. 


Equations 1 and 2 are the companies' demand function for labour and materials. $\theta_j$ is a labour productivity parameter.
The scaling parameters $\mu^{YL}_j$ and $\mu^{YM}_j$ describe how much of each variable companies will demand. The parameters are used to calibrate the model and therefore their only explanatory power is in determining sector input intensity. 

One can interpret the functions as the substitution effect from the parenthesis which looks at the relative price relation between wage/price of materials relative to the productivity & price for the output. The income effect can be seen from the output $Y_j$ so the more output being produced the more labour will be demanded.

The elasticity of substitution is $E^Y$ which states how substitutable the labour and materials inputs are. 

Equation 3 states there is perfect competition since the cost of the inputs equals the value of the output which means profits are 0.

Equation 4 is the demand function for materials the first subletter is the sector producing the good, the second letter is the one that consumes it. The materials demanded depens again on the substitution effect which depends on the relative prices between the price of the input good relative the price of the produced good. As usual, it also depends on the income effect which means the more materials that is demanded the more $x_{i,J}$ will be demanded.

Equation 5 states that the value of the materials of sector j equals the sum of the value of the input that sector i has produced which means the market clears.

Equation 6 is the consumption function where consumption depends on the substitution effect which is the relation between the price for the good relative to the price index of all goods, the price index also describes the relative prices between goods. The income effect depends positively on the output demanded, $Y^D$ and negatively one the price index. 

Equation 7 states that output demanded equals the value of the consumption.

Equation 8 states that output demanded equals the aggregated income derived from the number of people multiplied with their wage.

Equation 9 States that output of good i equals the consumption demanded plus the inputs that have been used to produce the goods. This forms the market clearing condition for output goods. 





In [3]:
importlib.reload(model_code_Final2)


<module 'modelproject' from 'C:\\Users\\Asker\\OneDrive - Københavns Universitet\\Documents\\Arbejde\\Undervisning\\INPA2\\Github\\Groups\\Inaugural-Project-2021_BSJH\\modelproject\\modelproject.py'>

## Setting parameters

Parameters for N, w, $E_C$, $E_M$, $E_Y$, $\Theta_1$ and $\Theta_2$ are considered as being given from the outside, while scale parameters needs to be calibrated to hit some data target. Here we set them arbitrarily at first. The wage is set to 1 as it functions as the numeraire. 

In [28]:
par=SimpleNamespace()
par.N=1000
par.w=1
par.EC=0.5
par.EM=0.5
par.EY=0.7
par.theta1=1
par.theta2=1
par.muYL1=0.5
par.muYL2=2
par.muYM1=0.3
par.muYM2=3
par.mux11=1
par.mux12=1
par.mux21=1
par.mux22=1
par.gamma1=1
par.gamma2=1
par.theta1m=1
par.theta2m=1


## Calibration

We use some mock data created by us that is supposed to symbolize actual data. As is standard for CGE models, all prices are calibrated around the value of 1. Following table describes the mock data for quantities given that calibration prices are set equal to 1:


| Variable | "Data Value" || Variable | "Data Value" |
| --- | --- | | --- | --- | 
| $L_1$ | 500 || $L_2$ | 500 |
| $Y_1$ | 750 || $Y_2$ | 750 |
| $M_1$ | 250 || $M_2$ | 250 |
| $x_{11}$ | 125 || $x_12$ | 125 |
| $x_{21}$ | 125 || $x_{22}$ | 125|
| $C_1$ | 500 || $C_2$ | 500 |
| $Y^D$ | 1500  ||



In [29]:
mock_data = np.array([500, 500,1,1, 750, 750, 250,250,1,1,125,125,125,125,500,500,1,1000])

As the model is not properly calibrated yet, the equations will not hold. If equations hold, then the following array would give all zeros:

In [30]:
model_code_Final2.h(mock_data,par)

array([  125., -1000.,    25., -2000.,     0.,     0.,  -125.,  -125.,
        -125.,  -125.,     0.,     0.,  -500.,  -500.,     0.,     0.,
           0.,     0.])

In [31]:
model_code_Final2.calibrate_2(mock_data, par)

Equation values before calibration: [  125. -1000.    25. -2000.     0.     0.  -125.  -125.  -125.  -125.
     0.     0.  -500.  -500.     0.     0.     0.     0.]
Equation values after calibration: [ 0.00000000e+00  0.00000000e+00 -2.84217094e-14  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00]


namespace(N=1000,
          w=1,
          EC=0.5,
          EM=0.5,
          EY=0.7,
          theta1=1,
          theta2=1,
          muYL1=0.6666666666666666,
          muYL2=0.6666666666666666,
          muYM1=0.33333333333333337,
          muYM2=0.3333333333333333,
          mux11=0.5,
          mux12=0.5,
          mux21=0.5,
          mux22=0.5,
          gamma1=0.5,
          gamma2=0.5,
          theta1m=1,
          theta2m=1)

In [32]:
par=model_code_Final2.calibrate_2(mock_data, par)

Equation values before calibration: [ 0.00000000e+00  0.00000000e+00 -2.84217094e-14  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
Equation values after calibration: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [33]:
par

namespace(N=1000,
          w=1,
          EC=0.5,
          EM=0.5,
          EY=0.7,
          theta1=1,
          theta2=1,
          muYL1=0.6666666666666666,
          muYL2=0.6666666666666666,
          muYM1=0.3333333333333333,
          muYM2=0.3333333333333333,
          mux11=0.5,
          mux12=0.5,
          mux21=0.5,
          mux22=0.5,
          gamma1=0.5,
          gamma2=0.5,
          theta1m=1,
          theta2m=1)

We then calibrate the model to hit the data as shown above:

In [26]:
par=model_code_Final2.calibrate(mock_data, par)

Equation values before calibration: [ 0.00000000e+00  0.00000000e+00 -2.84217094e-14  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
Equation values after calibration: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [27]:
par

namespace(N=1000,
          w=1,
          EC=0.5,
          EM=0.5,
          EY=0.7,
          theta1=1,
          theta2=1,
          muYL1=0.6666666666666666,
          muYL2=0.6666666666666666,
          muYM1=0.3333333333333333,
          muYM2=0.3333333333333333,
          mux11=0.5,
          mux12=0.5,
          mux21=0.5,
          mux22=0.5,
          gamma1=0.5,
          gamma2=0.5,
          theta1m=1,
          theta2m=1)

After calibration we can solve the model - without changing any parameters, this will simply give the mock-data values as shown above per the calibration effort

In [34]:
model_code_Final2.solve_model(mock_data, 0.5, 0.5, 0.7, 1, 1, par ,1 ,1)

array([ 500.,  500.,    1.,    1.,  750.,  750.,  250.,  250.,    1.,
          1.,  125.,  125.,  125.,  125.,  500.,  500.,    1., 1000.])

## Growth and the Central Algorithm

In order to illustrate model effects from productivity growth we consider the case where $\Theta_1$ grows at 3% pr. period and $\Theta_2$ grows at 1% pr. period giving sector 1 much higher productivity growth. 

The model solver needs an initialt point. While the mock-data functions well for small parameter changes, it results in the solver failing to find a solution for large changes. Thus for each period - we update the initial point as the solution from the previous period. A summary algorithm for solving the model across large different parameters for especially productivity is written here:


   1. Define Matrix with with equations
   2. Calibrate parameters to hit mock data
   3. Set Growth rates as for $\Theta_j$
   4. Solve model for $\Theta_j=1$ with the mock data as an initial point.
   5. Use the solution from 4. to solve for model with $\Theta_{j,t+1}=\Theta_{j,t}\cdot (1+g_j)$
   6. Repeat step 5. until the final period is reached.


This alghoritm is completely iterative as there is no "state variable" as capital in the model such that it is effectively a completely static model, where we just iteratively change the productivity parameter. 
    

## Results

Note that the simulations are not stable at elasticies equal to 1. Here the CES-function converges into a Cobb-Douglas form but this is not implemented.

In [35]:
interact(model_code_Final2.create_timeseries, EM1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5), 
         EY1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5), 
         x01=fixed(mock_data),EC1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5), parlist=fixed(par)) 

interactive(children=(FloatSlider(value=0.5, description='EC1', max=5.0, step=0.05), FloatSlider(value=0.5, de…

<function modelproject.create_timeseries(EC1, EM1, EY1, x01, parlist)>

Intuition: When one sector has a higher productivity growth rate than the other one could imagine that people would move towards the sector with the highest productivity growth rate where they are paid one’s marginal product. However, it is the opposite that actually happens.

Imagine a manufacturing sector and a service sector. Manufacturing has historically had much higher productivity growth rates than the service sector, but as time passes more people work in the service sector. The reason is that in order to attract people to the service sector the wage has to grow as fast or else there would be too big differences in the opportunity cost. Eventually because one sector becomes so effective, one barely needs any labour input and the sector can still produce an infinite amount of goods. In the figure above we see this effect happening.

Another interesting point is when one changes the elasticity of substitution between the goods in consumption which measures how freely consumers can substitute between each good. When the elasticity of substitution increases above 1, they will substitute more towards the good that has become relatively cheaper which comes from the sector with the highest productivity growth as goods the substitution effect starts to dominate the income effect. In the beginning the extra demand will lead to an increase of labour in the high productivity sector. However the effect of higher productivity growth rate will eventually dominate the former effect as one can see on the graph as time passes.  

Looking at $E^M$, the elasticity bewteen input materials, one sees that when $E^M$ increases the process of convergence slows down. The reason is that when sector 2 can substitute more easily away from labour inputs towards more input materials from sector 1, sector 2 will demand more goods from sector 1. In order to meet the increase in demand sector one has to hire more labour in order to meet the supply. The hiring of more labour in sector 1 will eventually be dominated by the effect from the higher productivity growth rates as we have seen before from the previous example.

Looking at $E^Y$, the elasticity bewteen labour and materials, one sees that when $E^Y$ increases the process of convergence slows down. Workers are paid the wage which is equal to their marginal product. As sector 1 becomes more productive the wage will increase. When the elasticity increases sector 1 wants to hire more labour as the labour is very productive but as the wages increases too fast the effect is that sector 1 will need less labour because a little labour input is so productive so again the productive sector will converge towards barely hiring any labour. But as the elasticity increases this effect will be delayed.

In [43]:
interact(model_code_Final2.graph_consumption, EM1=fixed(0.5), 
         EY1=fixed(0.5), 
         x01=fixed(mock_data),
         EC1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5), parlist=fixed(par))

interactive(children=(FloatSlider(value=0.5, description='EC1', max=5.0, step=0.05), Output()), _dom_classes=(…

<function modelproject.graph_consumption(EC1, EM1, EY1, x01, parlist)>

The graph depicts how much consumers are consuming of each good over time. As time goes sector 1 becomes more productive and can therefore supply more which means that the price of good 1 becomes relatively cheaper to good 2. Looking at the elasticity of substitution between the two goods one can see that the higher elasticity the more consumers will substitute towards the cheaper good which is good 1.


In [47]:
interact(model_code_Final2.create_timeseries_Y1_Y2, EM1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5), 
         EY1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5), 
         x01=fixed(mock_data),EC1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5), parlist=fixed(par)) 

interactive(children=(FloatSlider(value=0.5, description='EC1', max=5.0, step=0.05), FloatSlider(value=0.5, de…

<function modelproject.create_timeseries_Y1_Y2(EC1, EM1, EY1, x01, parlist)>

This graph depicts movements in output. We see that an increase in elasticity of substitution between materials and labour increases output for sector 1 as they can now produce cheaper comparatively by substituting towards the more effective labor component. By increasing elasticity of substitution between material inputs, sector 1 will also produce relatively more as they can use more of their own cheaper material input in production. 

In [38]:
interact(model_code_Final2.create_timeseries_L_M1, EM1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5), 
         EY1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5), 
         x01=fixed(mock_data),EC1=fixed(0.5), parlist=fixed(par)) 

interactive(children=(FloatSlider(value=0.5, description='EM1', max=5.0, step=0.05), FloatSlider(value=0.5, de…

<function modelproject.create_timeseries_L_M1(EC1, EM1, EY1, x01, parlist)>

    
Intuition: As $E^Y$, the elasticity of substitution between materials and labour, increases sector 1 will substitute towards labour because labour is becoming increasingly productive as time passes. For low values of the elasticity materials and labour are complimentary which mean that the sector needs both input. When inputs are easily substitutable the sector will hire labour because it is more productive.


In [39]:
interact(model_code_Final2.graph_x, EM1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5),
         EY1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5), 
         x01=fixed(mock_data),EC1=fixed(0.5), parlist=fixed(par)) 


interactive(children=(FloatSlider(value=0.5, description='EM1', max=5.0, step=0.05), FloatSlider(value=0.5, de…

<function modelproject.graph_x(EC1, EM1, EY1, x01, parlist)>

Intuition: As $E^M$, the elasticity of substitution between input materials from sector 1 or 2, increases both sectors will substitute towards inputs from sector 1 because as time passes sector 1's goods become cheaper since the sector becomes increasingly productive.



## Extension

In this section we consider the case, where there is also productivity growth in materials. This modifies equation 2 and 4 where $\Lambda_j$  is material productivity:

Material Aggregate demand

2. $$
\Lambda_j M_{j} = \mu_{j}^{YM}  \left( \frac {p_{j}^{M}} {p_{j}\Lambda_j} \right) ^{-E^Y} Y_{j}
$$

Division of aggregate materials

4. $$
x_{i,j} = \mu_{i, j}^{x} \left( \frac {p_{i}} {p_{j}^{M}} \right) ^{-E^M}\Lambda_j M_{j}
$$

Materials are assumed to grow at the same rates for the sectors as labour that is 3% pr. period in sector 1 and 1% pr. period in sector 2. 

We can now recheck how input demand between the material aggregate and labour changes for different levels of elasticity of substitution between labour and the material aggregate. 

In [48]:
interact(model_code_Final2.create_timeseries_L_M1_g, 
         EM1=fixed(0.1), 
         EY1=widgets.FloatSlider(min=0, max=5, step=0.05, value=0.5), 
         x01=fixed(mock_data),EC1=fixed(0.1), parlist=fixed(par));


interactive(children=(FloatSlider(value=0.5, description='EY1', max=5.0, step=0.05), Output()), _dom_classes=(…

As elasticity of substitution between the material aggregate and labour increases, they will substitute towards materials - as the supply is not fixed and thus becomes comparatively cheaper. This is different than what we saw above, where firms substituted away from materials as they became relatively less productive. 

Do note that this solution is very unstable and cannot be solved for many different elasticity combinations. 